# Junta os CSV

Junta todos os csv de um determinado diretório em um único csv

In [6]:
import pandas as pd
import os

In [7]:
def join_csv(pathFrom, fileTo, field='Fechamento'):
    counter = 0
    newCsv = pd.DataFrame()

    for file in os.listdir(pathFrom):
        counter = counter+1
        csv = pd.read_csv(pathFrom+'/'+file)
        
        # first column is Data
        if(counter == 1):
            newCsv['Data'] = csv['Data']

        name = os.path.splitext(file)[0]
        newCsv[name] = csv[field]

    newCsv.to_csv(fileTo)
    print('New file CSV successfully created')
    
def find_csv_max_lines(path):
    rows = []
    for file in os.listdir(path):
        with open(path+'/'+file) as f:
            lines = sum(1 for line in f)
            rows.append([file, lines])

    csv = pd.DataFrame(rows, columns=['File', 'Lines'])
    maxlines = csv['Lines'].max()
    return csv.loc[(csv['Lines'] == maxlines)]

def join_csv_merge(dfcsv, fileTo, path_old, path_older, list_choice=[]):
    newCsv = pd.DataFrame()
    define_data = False
    lenght = len(list_choice)
    
    for i, row_merge in merge.iterrows():
        name = os.path.splitext(row_merge.File)[0]
        if (lenght > 0 and name not in list_choice):
            continue
            
        fe1 = pd.read_csv(path_older+'/'+row_merge.File)
        fe2 = pd.read_csv(path_old+'/'+row_merge.File)
        dates=[]

        if (define_data == False):
            for i, row in fe1.iterrows():
                dates.append(row['Data'])
            for i, row in fe2.iterrows():
                dates.append(row['Data'])
            newCsv['Data'] = dates
            define_data = True

        rows=[]
        for i, row in fe1.iterrows():
            rows.append(row['Fechamento'])
        for i, row in fe2.iterrows():
            rows.append(row['Fechamento'])
        
        newCsv[name] = rows
        
    newCsv.to_csv(fileTo)

    print('New file CSV successfully created')

# De 1 diretório
#### Junta o preço de Fechamento de todos os csv em um único csv

In [13]:
# usando a função para juntar os csv's de 'datasets-b3'
join_csv('datasets-b3', 'datasets/data.csv')

New file CSV successfully created


# De 2 diretórios
#### Junta o preço de Fechamento de todos os csv de 2 diretório em um único csv
Somente os csv que existem nos dois diretórios

In [8]:
csv19 = find_csv_max_lines('datasets-b3-2019')
csv20 = find_csv_max_lines('datasets-b3')
csv20.shape, csv19.shape

((264, 2), (204, 2))

In [14]:
merge = pd.merge(csv20, csv19, left_on='File', right_on='File')
merge.shape

(201, 3)

### De todas as ações

In [13]:
join_csv_merge(merge, 'datasets/data.csv', 'datasets-b3', 'datasets-b3-2019')

New file CSV successfully created


### Somente de algumas ações

In [35]:
acoes = pd.read_csv('datasets/composicao_indice.csv', names=['Código'])['Código']
acoes

0     ABEV3
1     AZUL4
2     B3SA3
3     BBAS3
4     BBDC3
      ...  
70    VALE3
71    VIVT4
72    VVAR3
73    WEGE3
74    YDUQ3
Name: Código, Length: 75, dtype: object

In [41]:
list_acoes = ['AALR3', 'AAPL34', 'ABCB4']
join_csv_merge(merge, 'datasets/data_cart.csv', 'datasets-b3', 'datasets-b3-2019', acoes.tolist())

New file CSV successfully created
